## Multilayer Perceptron with Tensorflow

In [1]:
import tensorflow as tf

In [2]:
# Creating Interactive Session to run on the fly
sess = tf.InteractiveSession()

In [3]:
# Creating Placeholder
x = tf.placeholder(tf.float32, shape=[None, 784]) # Input Data
y_ = tf.placeholder(tf.float32, shape=[None, 10])  # Output Class

### Assigning Weights and Bias to Tensors

In [4]:
# Weight Tensor
w = tf.Variable(tf.zeros([784, 10], tf.float32))

# Bias Tensor
b = tf.Variable(tf.zeros([10], tf.float32))

In [5]:
# Execute the assignment Operation
sess.run(tf.initialize_all_variables())

Instructions for updating:
Use `tf.global_variables_initializer` instead.


### Softmax Regression

In [6]:
y = tf.nn.softmax(tf.matmul(x, w) + b)

### Cost Function

In [7]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

### Type of Optimization: Gradient Descent

In [8]:
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

### Training Batches

In [9]:
# Load MNIST Dataset
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [10]:
# Train using mini batches
for i in range(1000):
    batch = mnist.train.next_batch(50)
    train_step.run(feed_dict={x: batch[0], y_:batch[1]})

### Test the Model

In [11]:
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
acc = accuracy.eval(feed_dict={x: mnist.test.images, y_:mnist.test.labels})*100
print "The Final Accuracy for Simple ANN Model is: ", format(acc)
sess.close()

The Final Accuracy for Simple ANN Model is:  90.9200012684


## CNN with Tensorflow

In [12]:
import tensorflow as tf

# Close already open session
sess.close()

# Open a new Interactive session
sess = tf.InteractiveSession()

In [13]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


### Initial parameter

In [14]:
# Creating Placeholder
x = tf.placeholder(tf.float32, shape=[None, 784]) # Input Data
y_ = tf.placeholder(tf.float32, shape=[None, 10])  # Output Class

### Weights

In [15]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

### Bias

In [16]:
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

### Convolutional Layes

In [17]:
def conv2d(x, w):
    return tf.nn.conv2d(x, w, strides=[1, 1, 1, 1], padding='SAME')

### Max Pooling

In [18]:
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

### First Convolutional and Max-Pooling Layers

In [19]:
# Weights and Biases
w_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

In [20]:
# Converting Image Dataset to Tensors
x_image = tf.reshape(x, [-1, 28, 28, 1])

In [21]:
# Convolve with weight tensor and add biases
convolve1 = conv2d(x_image, w_conv1) + b_conv1

In [22]:
# Apply the Relu activation function
h_conv1 = tf.nn.relu(convolve1)

# Apply the Max Pooling
h_pool1 = max_pool_2x2(h_conv1)

In [23]:
# First layer Completed
layer1 = h_pool1

### Second Convolutional and Max-Pooling Layer

In [24]:
# Weights and Biases
w_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

# Converting Image Dataset to Tensors
# Already Converted

# Convolve with weight tensor and add biases
convolve2 = conv2d(layer1, w_conv2) + b_conv2

# Apply the Relu activation function
h_conv2 = tf.nn.relu(convolve2)

# Apply the Max Pooling
h_pool2 = max_pool_2x2(h_conv2)

# Second layer Completed
layer2 = h_pool2

### Third Convolutional and Max-Pooling Layer

In [25]:
# Apply fully connected layer
w_fc1 = weight_variable([7*7*64, 1024])
b_fc1 = bias_variable([1024])

# Flatenning Second layer
layer2_matrix = tf.reshape(layer2, [-1, 7*7*64])

# Matrix Multiplication - Applying Weights and Bias
matmul1_fc1 = tf.matmul(layer2_matrix, w_fc1) + b_fc1

# Apply the Relu activation function
h_fc1 = tf.nn.relu(matmul1_fc1)

# Third layer Completed
layer3 = h_fc1

### Optional Phase of Dropout

In [26]:
# Apply Dropout
keep_prob = tf.placeholder(tf.float32)
layer3_drop = tf.nn.dropout(layer3, keep_prob)

### Final Layer -Readout Layer

In [27]:
# Weights and Bias - 1024 neurons and 10 Output Features
w_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

# Matrix Multiplication
matmul_fc2 = tf.matmul(layer3_drop, w_fc2) + b_fc2

# Apply Softmax Activation Function
y_conv = tf.nn.softmax(matmul_fc2)

# Layer3 Completed
layer4 = y_conv

### Summary of Deep Convolutional Neural Network

1. Input - MNIST dataset
2. Convolutional and Max-Pooling
3. Convolutional and Max-Pooling
4. Fully Connected Layer
5. Processing - Dropout
6. Readout Layer - Fully Connected Layer
7. Outputs - Calssified Digits

### Define Functions and Train Model

In [28]:
# Define the Loss Function
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_*tf.log(layer4), reduction_indices=[1]))

# Define the Optimizer
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

# Define the Prediction
correct_prediction = tf.equal(tf.argmax(layer4, 1), tf.argmax(y_, 1))

# Define Accuracy
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [35]:
# Run the Session
sess.run(tf.global_variables_initializer())

In [37]:
# Finally do the training - It might take a while
for i in range(100):
    batch = mnist.train.next_batch(50)
    if i%100== 0:
        train_accuracy = accuracy.eval(feed_dict={x:batch[0], y_:batch[1], keep_prob:1.0})
        print "Step %d, Training Accuracy: %g"%(i, float(train_accuracy))
    train_step.run(feed_dict={x:batch[0], y_:batch[1], keep_prob:0.5})       


Step 0, Training Accuracy: 0.72


In [39]:
# Test Accuracy
print "Testing Accuracy: %g"%(accuracy.eval(feed_dict={x: mnist.test.images, y_:mnist.test.labels}))

InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder_4' with dtype float
	 [[Node: Placeholder_4 = Placeholder[dtype=DT_FLOAT, shape=[], _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op u'Placeholder_4', defined at:
  File "/usr/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python2.7/dist-packages/traitlets/config/application.py", line 596, in launch_instance
    app.start()
  File "/usr/local/lib/python2.7/dist-packages/ipykernel/kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  File "/usr/local/lib/python2.7/dist-packages/zmq/eventloop/ioloop.py", line 162, in start
    super(ZMQIOLoop, self).start()
  File "/usr/local/lib/python2.7/dis

In [ ]:
# Close the Session
sess.close()